In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
using TrajOptPlots
const MOI = MathOptInterface
using Random

  Activating environment at `~/Documents/planar_quadruped_landing/Project.toml`


In [2]:
include("quadratic_cost.jl")
include("planar_quadruped.jl")
include("sparseblocks.jl")
include("utils.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")
include("ref_traj.jl")

WebIO._IJuliaInit()

reference_trajectory

## Reference Trajectory

## Problem Definition

In [3]:
# Dynamics model
model = PlanarQuadruped()
g, mb, lb, l1, l2 = model.g, model.mb, model.lb, model.l1, model.l2

# Discretization
tf = 0.6
dt = 0.02
N = Int(ceil(tf/dt)) + 1
times = range(0,tf, length=N)
t_trans = 0.2

n = 14
m = 4

# Initial Conditions
# currently, we assume the initial mode ID is 1
# xinit = [-0.4;0.6;-3.1415/9;        0.0;0.0; -1.0;0.2; 0.0;-0.5;0.0; 0.0;0.0; 0.0;-0.5]
# xterm = [-lb/2;sqrt(l1^2+l2^2);0.0; 0.0;0.0; -lb;0.0;  0.0;0.0;0.0;  0.0;0.0; 0.0;0.0]
xinit = zeros(n)
xinit[1] = -lb/2                  # xb
xinit[2] = sqrt(l1^2+l2^2) + 0.05 # yb
xinit[3] = 10 * pi / 180          # theta
xinit[6] = -lb                    # x2
xinit[7] = 0.05                   # y2
xinit[9] = -3.0                   # yb_dot
xinit[12] = -3.0                  # y2_dot

xterm = zeros(n)
xterm[1] = -lb/2           # xb
xterm[2] = sqrt(l1^2+l2^2) # yb
xterm[6] = -lb             # x2

init_mode = 1

# Reference Trajectory
Xref, Uref = reference_trajectory(model, times, t_trans, xinit, xterm, init_mode)

# Objective
Random.seed!(1)
Q = Diagonal([1.0;1.0;1.0; 1.0;1.0; 1.0;1.0; 1.0;1.0;1.0; 1.0;1.0; 1.0;1.0])
R = Diagonal(fill(1e-3,4))
Qf = Q

obj = map(1:N-1) do k
    LQRCost(Q,R,Xref[k],Uref[k])
end
push!(obj, LQRCost(Qf, R*0, Xref[N], Uref[1]))

# Define the NLP
nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end]);

## Solve

In [4]:
nlp.cinds

5-element Vector{UnitRange{Int64}}:
 1:14
 15:28
 29:448
 449:479
 480:500

In [5]:
# Initial guess
xtransit = zeros(n)
xtransit[1] = -lb/2 # xb
xtransit[2] = l1    # yb
xtransit[3] = 0     # theta
xtransit[6] = -lb   # x2

# Random.seed!(1)
# Uguess = [u + 0.1*randn(length(u)) for u in Uref]
# Xguess = [x + 0.1*randn(length(x)) for x in Xref]

Uguess = [zeros(length(u)) for u in Uref]
Xguess = [zeros(length(x)) for x in Xref]

timesteps_phase_1 = nlp.k_trans

for i = 1:timesteps_phase_1
    Xguess[i] = xinit + (xtransit - xinit)/timesteps_phase_1 * i
end

# display((xtransit - xinit)/timesteps_phase_1)

In [6]:
Z0 = packZ(nlp, Xguess, Uguess);
# nlp = HybridNLP(model, obj, init_mode, tf, N, Xref[1], Xref[end], use_sparse_jacobian=false);

# Z_sol, solver = solve(Z0, nlp, c_tol=1e-6, tol=1e-6)
Z_sol, solver = solve(Z0, nlp, c_tol=1e-4, tol=1e-2)

Creating NLP Block Data...
Creating Ipopt...
Adding constraints...
starting Ipopt Solve...

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:   277000
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      554
                     variables with only lower bounds:       93
                variables with lower and upper bounds:    

([-0.24999999833772013, 0.4035534520716734, 0.17453292512236723, -5.911118365939537e-10, 4.692409784284352e-8, -0.5000000003698846, 0.04999999998361138, 9.980701286946877e-11, -2.999999980898347, -1.6835822986299908e-11  …  1.8718249965293496e-8, -0.499999999244163, 2.2556878818588898e-8, 4.938458702588705e-10, 1.1275448736841334e-9, -2.7374707149872375e-11, -4.443594686442027e-11, -1.3678627895883782e-10, -1.1667151529654125e-10, -2.257289372960156e-9], Ipopt.Optimizer)

In [7]:
Z_sol[1:14] - xinit

14-element Vector{Float64}:
  1.6622798693077812e-9
  6.147839964887325e-8
 -7.706571492072101e-11
 -5.911118365939537e-10
  4.692409784284352e-8
 -3.6988456741937625e-10
 -1.6388619628049383e-11
  9.980701286946877e-11
  1.910165314811252e-8
 -1.6835822986299908e-11
  4.199556529017935e-11
  7.19147319472313e-9
 -1.3819512912337204e-11
 -1.0160273215970499e-10

In [8]:
Z_sol[end-13:end] - xterm

14-element Vector{Float64}:
 -1.9126585315554223e-9
 -6.139759450851656e-8
  7.577089961220423e-11
  4.5692056331898117e-10
  1.8718249965293496e-8
  7.558370040960938e-10
  2.2556878818588898e-8
  4.938458702588705e-10
  1.1275448736841334e-9
 -2.7374707149872375e-11
 -4.443594686442027e-11
 -1.3678627895883782e-10
 -1.1667151529654125e-10
 -2.257289372960156e-9

EXIT: Restoration failed!

EXIT: Converged to a point of local infeasibility. Problem may be infeasible.

In [9]:
# # display(Z_sol)
# using DelimitedFiles
# using CSV
# writedlm( "data.csv",  Z_sol, ',')